In [1]:
import numpy as np
from astropy.io import ascii
from pylab import *
from astropy.table import Table, vstack
import pandas as pd
from expecto import get_spectrum
from scipy.interpolate import RegularGridInterpolator as rgi
from itertools import product
import pysynphot
#%run Interpolate.ipynb
#%run GetSpectra.ipynb
#%run Binning.ipynb

/Users/tchawla/anaconda3/lib/python3.10/site-packages/pysynphot/locations.py:345: UserWarning: Extinction files not found in /Users/tchawla/Downloads/grp/redcat/trds/extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))


In [2]:
'''

'''

def normalize_expecto_to_magnitude(expecto_wave, expecto_flux, magnitude):

    expecto_flux /= 10**8 #to ergs/s/cm^2/Ang
    h = pysynphot.units.H # ergs * s (plank's constant)
    c = pysynphot.units.C #meters/second (speed of light)
    c *= 10**10 #Angstroms/second

    #converts ergs (energy measurement) to photon count
    ergs_to_photons = h*c/expecto_wave

    expecto_flux /= ergs_to_photons

    sp = pysynphot.ArraySpectrum(expecto_wave.value, expecto_flux.value)
    sp_norm = sp.renorm(magnitude, 'vegamag', pysynphot.ObsBandpass('2mass,j'))
    

    return sp_norm.wave, sp_norm.flux

'''
if __name__ == "__main__":

    expecto_spectrum = expecto.get_spectrum(T_eff = 5800, log_g = 4.5, cache = False) #units are ergs/s/cm^2/cm

    expecto_flux = expecto_spectrum.flux
    expecto_wave = expecto_spectrum.wavelength

    wave, flux = normalize_expecto_to_magnitude(expecto_wave, expecto_flux, 10)
    plt.plot(wave, flux)
    plt.yscale('log')
    plt.show()
    
'''


'\nif __name__ == "__main__":\n\n    expecto_spectrum = expecto.get_spectrum(T_eff = 5800, log_g = 4.5, cache = False) #units are ergs/s/cm^2/cm\n\n    expecto_flux = expecto_spectrum.flux\n    expecto_wave = expecto_spectrum.wavelength\n\n    wave, flux = normalize_expecto_to_magnitude(expecto_wave, expecto_flux, 10)\n    plt.plot(wave, flux)\n    plt.yscale(\'log\')\n    plt.show()\n    \n'

In [3]:
'''
function to get spectrum from expecto
'''

def spectrum(T, mag, surf_g=4.5):
    spectrum = get_spectrum(T_eff=T, log_g=surf_g, cache=False)
    
    wave, flux = normalize_expecto_to_magnitude(spectrum.wavelength, spectrum.flux, mag)
    
    return wave, flux

In [6]:
'''
function that takes in spectrum and wavelength range, outputs flux and wavelength. 

This version takes in a wavelength range

np.where returns a nested array, to retrieve integer, need to index it at 0 twice.
'''

def spec_wave_range(spectrum, wave_in, wave_out):
    wave_in_converted = wave_in*10000
    wave_out_converted = wave_out*10000
    wave_in_index = np.where(spectrum[0] == wave_in_converted)[0]
    wave_out_index = np.where(spectrum[0] == wave_out_converted)[0]
    wavelength = spectrum[0][wave_in_index[0]:wave_out_index[0]]
    flux = spectrum[1][wave_in_index[0]:wave_out_index[0]]
    return wavelength, flux

In [ ]:
'''
Function that takes spectrum, wavelength, and water content as inputs. Interpolates transmission values based on 
water content and wavelength. Multiplies stellar spectrum with transmission to simulate atmospheric effects from pwv.

wavelength range is limited from .6 (inclusive) to 2.4 (non-inclusive).

water vapor range is limited from 1 to 5 (inclusive).

These are limitations from SKYCALC files read in, could be adjusted with different data.

Instead of iterating through every wavelength value to multiply, it passes a list of wavelength values to interpolating 
function, and multiplies full array simultaneously.
'''

def spectra_transmission(spectrum, wave_in, wave_out, water_content):
    wave_range = spec_wave_range(spectrum, wave_in, wave_out)[0]
    flux_range = spec_wave_range(spectrum, wave_in, wave_out)[1]
    #print(wave_range)
    plt.plot(wave_range, flux_range)
    plt.show()

    wave_range_adjusted = wave_range/10
    
    transmission = interpolate(water_content, wave_range_adjusted)
    transmission_flux = transmission*flux_range
    plt.plot(wave_range, transmission_flux)
    plt.yscale("log")
    plt.show()